In [1]:
import os
from dotenv import load_dotenv
import sounddevice as sd
import soundfile as sf
from scipy.io.wavfile import write
import json
import numpy as np
import streamlit as st
from typing import Dict, Any, List, Tuple
from pymongo import MongoClient
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from groq import Groq
import time
from complex_helper import *

load_dotenv()

# Initialize API clients
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")

# Initialize Groq client and LLM
client = Groq(api_key=GROQ_API_KEY)
llm = ChatGroq(api_key=GROQ_API_KEY, model="llama-3.1-8b-instant", temperature=0)

In [2]:
interviewr_bot = InterviewerBot()

In [3]:
candidate_data = interviewr_bot.extract_candidate_info()
candidate_data

{'_id': 'JOHDOE-20241013151130',
 'id': 'JOHDOE-20241013151130',
 'personal_information': {'first_name': 'John',
  'last_name': 'Doe',
  'date_of_birth': datetime.datetime(1990, 1, 1, 0, 0),
  'email': 'john.doe@example.com',
  'phone_number': '1234567890',
  'address': '123 Main St'},
 'education_qualifications': {'highest_education_level': "Bachelor's Degree",
  'university_name': 'XYZ University',
  'degree_name': 'B.Sc in Computer Science',
  'specialization': 'Computer Science and Engineering',
  'percentage_marks': 8.5,
  'certificates_diplomas': ['AWS Certified Developer']},
 'work_experience': [{'company_name': 'ABC Corp',
   'job_title': 'Software Engineer',
   'start_date': datetime.datetime(2018, 1, 1, 0, 0),
   'end_date': datetime.datetime(2021, 12, 31, 0, 0),
   'key_responsibilities': ['Develop software', 'Maintain systems'],
   'achievements': ['Employee of the Month']}],
 'technical_skills': {'programming_languages': ['Python', 'Java'],
  'development_tools': ['VS Code

In [11]:
candidate_data['personal_information']['first_name']

'John'

In [4]:
greet_and_question = interviewr_bot.generate_questions()
greet_and_question


{'interview': {'greeting_script': "Hello John Doe, it's great to have you here today. I must say, I'm impressed with your experience as a Software Engineer at ABC Corp, where you were recognized as Employee of the Month. Your achievements in developing software and maintaining systems are truly commendable. I'm excited to learn more about your skills and experience in the field of Data Science and AI.",
  'questions': ['Can you walk me through your experience with Python and Java programming languages? How have you applied them in your previous roles?',
   "You've mentioned 'Project A' and 'Project B' as some of your notable projects. Can you tell me more about these projects, including your role, the technologies used, and the outcomes?",
   'A common challenge in Data Science is dealing with imbalanced datasets. Can you describe a time when you encountered this issue and how you approached it?',
   "As a team player, can you give an example of a successful collaboration you've been a

In [12]:
greet_and_question['interview']['greeting_script']

"Hello John Doe, it's great to have you here today. I must say, I'm impressed with your experience as a Software Engineer at ABC Corp, where you were recognized as Employee of the Month. Your achievements in developing software and maintaining systems are truly commendable. I'm excited to learn more about your skills and experience in the field of Data Science and AI."

In [5]:
greetings = greet_and_question['interview']['greeting_script']
questions = greet_and_question['interview']['questions']
questions

['Can you walk me through your experience with Python and Java programming languages? How have you applied them in your previous roles?',
 "You've mentioned 'Project A' and 'Project B' as some of your notable projects. Can you tell me more about these projects, including your role, the technologies used, and the outcomes?",
 'A common challenge in Data Science is dealing with imbalanced datasets. Can you describe a time when you encountered this issue and how you approached it?',
 "As a team player, can you give an example of a successful collaboration you've been a part of? How did you communicate with your team and what was your role in the project?",
 "What are your long-term goals in the field of Data Science and AI? Are there any specific areas you'd like to specialize in or any new skills you'd like to acquire?"]

In [6]:
# score = {}
# follow_up_questions_1={}

# for i in questions[:2]:
#     print('First Question Ready To Serve You..')
#     question_speech = interviewr_bot.convert_question_to_speech(i)
#     print('Answer the Question')
#     answer_speech = interviewr_bot.record_until_enter()
#     print('Converting Speech Into Text For Better Understanding')
#     answer_text = interviewr_bot.convert_speech_to_text()
#     print('Analyzing the answer you provided')
#     evaluate_answer = interviewr_bot.evaluate_answer(i, answer_text)
#     score[i] = {
#         "question_speech": question_speech,
#         "answer_speech": answer_speech,
#         "answer_text": answer_text,
#         "evaluation": evaluate_answer
#     }
#     print('store answer in my record')
#     if score[i]['evaluation']['evaluation']['score'] <= 6:
#         print('generating followup question for better understanding about what you say')
#         follow_up_questions = interviewr_bot.generate_follow_up_question(i,answer_text)
#         print('here the followup question', follow_up_questions)
#         question_speech = interviewr_bot.convert_question_to_speech(i)
#         print('can you clarify the answer with more detail way')
#         answer_speech = interviewr_bot.record_until_enter()
#         print('understanding your answer')
#         answer_text = interviewr_bot.convert_speech_to_text()
#         print('evaluating your answer')
#         evaluate_answer = interviewr_bot.evaluate_answer(i, answer_text)
#         print('adding your answer to my knowldege source')
#         score[i] = {
#             "follow_up_question_1":follow_up_questions,
#             'follow_up_question_answer_1': answer_text,
#             'evaluation_answer_1': evaluate_answer,         
#         }
#         if score[i]['evaluation']['evaluation']['score'] <= 6:
#             print('i fill your answer is still not completed can you explain more')
#             follow_up_questions = interviewr_bot.generate_follow_up_question(i,answer_text)
#             print('exaplain me this question', follow_up_questions)
#             question_speech = interviewr_bot.convert_question_to_speech(i)
#             print('please answer with more clarity')
#             answer_speech = interviewr_bot.record_until_enter()
#             print('converting speech to text')
#             answer_text = interviewr_bot.convert_speech_to_text()
#             print('storing your asnwer into my knowledge source')
#             evaluate_answer = interviewr_bot.evaluate_answer(i, answer_text)
#             score[i] = {
#                 "follow_up_question_2":follow_up_questions,
#                 'follow_up_question_answer_2': answer_text,
#                 'evaluation_answer_2': evaluate_answer,         
#             }
#         else:
#             pass
#     else:
#         pass

In [7]:
# score

In [8]:
score = {}

for idx, question in enumerate(questions[:2], 1):
    print(f"\n--- [Question {idx}] Serving Initial Question ---")
    question_speech = interviewr_bot.convert_question_to_speech(question)
    
    print(f"[Question {idx}] Awaiting User Answer (Press Enter to Stop Recording)...")
    answer_speech = interviewr_bot.record_until_enter()
    
    print(f"[Question {idx}] Converting Speech to Text...")
    answer_text = interviewr_bot.convert_speech_to_text()
    
    print(f"[Question {idx}] Evaluating Initial Answer...")
    evaluation = interviewr_bot.evaluate_answer(question, answer_text)
    
    score[question] = {
        "question_speech": question_speech,
        "answer_speech": answer_speech,
        "answer_text": answer_text,
        "evaluation": evaluation
    }

    if evaluation['evaluation']['score'] <= 6:
        print(f"[Question {idx}] Score ≤ 6 — Generating Follow-Up Question 1...")
        follow_up_question_1 = interviewr_bot.generate_follow_up_question(question, answer_text)
        print(f"[Question {idx}] Follow-Up Question 1: {follow_up_question_1}")
        
        follow_up_q1_speech = interviewr_bot.convert_question_to_speech(follow_up_question_1)
        print(f"[Question {idx}] Awaiting Clarification Answer (Follow-Up 1)...")
        follow_up_a1_speech = interviewr_bot.record_until_enter()
        
        print(f"[Question {idx}] Converting Follow-Up 1 Answer to Text...")
        follow_up_a1_text = interviewr_bot.convert_speech_to_text()
        
        print(f"[Question {idx}] Evaluating Follow-Up 1 Answer...")
        evaluation_follow_up_1 = interviewr_bot.evaluate_answer(follow_up_question_1, follow_up_a1_text)
        
        score[question].update({
            "follow_up_question_1": follow_up_question_1,
            "follow_up_question_1_speech": follow_up_q1_speech,
            "follow_up_answer_1_speech": follow_up_a1_speech,
            "follow_up_answer_1_text": follow_up_a1_text,
            "evaluation_follow_up_1": evaluation_follow_up_1
        })

        if evaluation_follow_up_1['evaluation']['score'] <= 6:
            print(f"[Question {idx}] Score Still ≤ 6 — Generating Follow-Up Question 2...")
            follow_up_question_2 = interviewr_bot.generate_follow_up_question(question, follow_up_a1_text)
            print(f"[Question {idx}] Follow-Up Question 2: {follow_up_question_2}")
            
            follow_up_q2_speech = interviewr_bot.convert_question_to_speech(follow_up_question_2)
            print(f"[Question {idx}] Awaiting Clarification Answer (Follow-Up 2)...")
            follow_up_a2_speech = interviewr_bot.record_until_enter()
            
            print(f"[Question {idx}] Converting Follow-Up 2 Answer to Text...")
            follow_up_a2_text = interviewr_bot.convert_speech_to_text()
            
            print(f"[Question {idx}] Evaluating Follow-Up 2 Answer...")
            evaluation_follow_up_2 = interviewr_bot.evaluate_answer(follow_up_question_2, follow_up_a2_text)
            
            score[question].update({
                "follow_up_question_2": follow_up_question_2,
                "follow_up_question_2_speech": follow_up_q2_speech,
                "follow_up_answer_2_speech": follow_up_a2_speech,
                "follow_up_answer_2_text": follow_up_a2_text,
                "evaluation_follow_up_2": evaluation_follow_up_2
            })
        else:
            print(f"[Question {idx}] Follow-Up 1 Clarified Well — No Need for Follow-Up 2.")
    else:
        print(f"[Question {idx}] Good Score — No Follow-Up Needed.")

print("\n--- Interview Session Complete ---")



--- [Question 1] Serving Initial Question ---


BadRequestError: Error code: 400 - {'error': {'message': 'The model `playai-tts` requires terms acceptance. Please have the org admin accept the terms at https://console.groq.com/playground?model=playai-tts', 'type': 'invalid_request_error', 'code': 'model_terms_required'}}